In [49]:
import datetime
import json
import os
import mthree
import numpy as np
from qiskit import IBMQ
from qiskit_experiments.framework import ExperimentEncoder
from qiskit_research.mzm_generation import (
    KitaevHamiltonianExperiment,
    KitaevHamiltonianExperimentParameters,
)
from qiskit_research.mzm_generation.utils import transpile_circuit
from qiskit_research.mzm_generation.utils import pick_qubit_layout, orbital_combinations, get_backend

from qiskit.circuit.library import RZGate
from qiskit_research.utils.dynamical_decoupling import get_instruction_durations

from qiskit.providers.aer import AerSimulator
from qiskit.providers.aer.noise import NoiseModel

from qiskit.transpiler import CouplingMap, InstructionDurations

In [51]:
# get backend

# uncomment to use hardware backend
# IBMQ.load_account()
# provider = IBMQ.get_provider(hub='ibm-q-internal', group='deployed', project='default')
# backend_name = 'ibmq_guadalupe'

# Aer simulator backend
IBMQ.load_account()
simulating_provider = IBMQ.get_provider(hub='ibm-q-internal', group='deployed', project='default')
provider = None
backend_name = "aer_simulator"
simulating_backend_name = 'ibmq_guadalupe'

ibmqfactory.load_account:WARNING:2022-07-07 14:37:05,649: Credentials are already in use. The existing account in the session will be replaced.


In [52]:
# experiment parameters
n_modes = 6
tunneling_values = [-1.0]
superconducting_values = [1.0]
chemical_potential_values = list(np.linspace(0.0, 3.0, num=5))
occupied_orbitals_list = list(orbital_combinations(n_modes, threshold=2))
# dynamical_decoupling_sequences = None
dynamical_decoupling_sequences=[None, 'X2pm', 'XY4pm', 'XY8pm']
num_dd_passes=1
uhrig_spacing = False
concat_layers = 1
shots = 10000
readout_calibration_shots = 10000

# pick qubits
qubits, _, error_score = pick_qubit_layout(n_modes, simulating_backend_name, simulating_provider)
print(f"Using qubits {qubits}.")

# date
date = datetime.datetime.now().isoformat()

# create experiment
params = KitaevHamiltonianExperimentParameters(
    timestamp=date,
    backend_name=backend_name,
    qubits=qubits,
    n_modes=n_modes,
    tunneling_values=tunneling_values,
    superconducting_values=superconducting_values,
    chemical_potential_values=chemical_potential_values,
    occupied_orbitals_list=occupied_orbitals_list,
    dynamical_decoupling_sequences=dynamical_decoupling_sequences,
    num_dd_passes=num_dd_passes,
    uhrig_spacing=uhrig_spacing,
    concat_layers=concat_layers,
    seed=11364,
)
experiment = KitaevHamiltonianExperiment(params, simulating_provider)

print(f"Total number of circuits: {len(experiment.circuits())}")

Using qubits [12, 13, 14, 11, 8, 5].


/Users/georgezhou/miniforge3/envs/DDResearch/lib/python3.9/site-packages/qiskit/providers/aer/noise/device/models.py:264: UserWarning: Device model returned an invalid T_2 relaxation time greater than the theoretical maximum value 2 * T_1 (inf > 2 * 86800.27280859138). Truncating to maximum value.
  warn("Device model returned an invalid T_2 relaxation time greater than"
/Users/georgezhou/miniforge3/envs/DDResearch/lib/python3.9/site-packages/qiskit/providers/aer/noise/device/models.py:264: UserWarning: Device model returned an invalid T_2 relaxation time greater than the theoretical maximum value 2 * T_1 (inf > 2 * 86800.27280859138). Truncating to maximum value.
  warn("Device model returned an invalid T_2 relaxation time greater than"
/Users/georgezhou/miniforge3/envs/DDResearch/lib/python3.9/site-packages/qiskit/providers/aer/noise/device/models.py:264: UserWarning: Device model returned an invalid T_2 relaxation time greater than the theoretical maximum value 2 * T_1 (inf > 2 * 86

Total number of circuits: 1560


In [44]:
%%time

# schedule readout calibration
print("Scheduling readout calibration ...")
mit = mthree.M3Mitigation(experiment.backend)
mit.cals_from_system(qubits, shots=readout_calibration_shots, async_cal=True)

# schedule experiment jobs
print(f"Scheduling experiment ...")
# limit number of circuits per job to avoid timeouts
experiment.backend.configuration().max_experiments = 300
data = experiment.run(shots=shots)
if provider:
    # this only works if the Qiskit Experiments database service is available
    data.save()

# save readout calibration
filename = f"data/readout_calibration/{params.backend_name}/{date}.json"
os.makedirs(os.path.dirname(filename), exist_ok=True)
print(f"Waiting for readout calibration data ...")
mit.cals_to_file(filename)
print(f"Readout calibration saved.")

Scheduling readout calibration ...
Scheduling experiment ...
Waiting for readout calibration data ...
Readout calibration saved.
CPU times: user 13.8 s, sys: 718 ms, total: 14.5 s
Wall time: 14 s


In [45]:
data.block_for_results()

filename = f"data/{params.backend_name}/{params.filename}.json"
os.makedirs(os.path.dirname(filename), exist_ok=True)
with open(filename, "w") as f:
    json.dump(data, f, cls=ExperimentEncoder)

print(params.filename)

2022-07-05T14:30:57.441035_aer_simulator_n6
